<a href="https://colab.research.google.com/github/Driggsz/CampBrasileiro/blob/main/Brasileirao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [4]:
# Carregar os dados
df = pd.read_csv('/Classificacao_2023.csv')

In [5]:
# Dividir os dados em treinamento (até rodada 30) e teste (rodadas 31 a 38)
dados_treinamento = df[df['Rodada'] <= 30]
dados_teste = df[df['Rodada'] > 30]


In [6]:
# Selecionar as features e o target para treinamento
X_treinamento = dados_treinamento.drop(['Time', 'Temporada', 'Posicao', 'Rodada'], axis=1)
y_treinamento = dados_treinamento['Posicao']

In [ ]:
# Treinar o modelo
modelo = LogisticRegression()
modelo.fit(X_treinamento, y_treinamento)


In [8]:
# Avaliar a precisão do modelo nos dados de treinamento
previsao_treinamento = modelo.predict(X_treinamento)
acuracia_treinamento = accuracy_score(y_treinamento, previsao_treinamento)
print("Acurácia do modelo nos dados de treinamento:", acuracia_treinamento)

Acurácia do modelo nos dados de treinamento: 0.45666666666666667


In [9]:
# Selecionar as features para previsão das 8 rodadas restantes
X_teste = dados_teste.drop(['Time', 'Temporada', 'Posicao', 'Rodada'], axis=1)



In [10]:
# Prever a posição de cada time na 38ª rodada
posicoes_previstas_38_rodada = modelo.predict(X_teste)

In [11]:
# Mapear os números das posições de volta aos nomes dos times
posicoes_previstas_38_rodada_nomes = dados_teste['Time'].values

In [12]:
# Criar um DataFrame com as previsões da colocação
df_previsao = pd.DataFrame({'Time': posicoes_previstas_38_rodada_nomes, 'Posicao_Previsao': posicoes_previstas_38_rodada})

In [13]:


# Agrupar por time e calcular a posição prevista mais frequente
df_previsao_agrupada = df_previsao.groupby('Time')['Posicao_Previsao'].agg(lambda x:x.value_counts().index[0]).reset_index()




In [14]:
# Agrupar por time e calcular a posição prevista mais frequente
df_previsao_agrupada = df_previsao.groupby('Time')['Posicao_Previsao'].agg(lambda x:x.value_counts().index[0]).reset_index()

In [15]:
# Ordenar o DataFrame pela posição prevista
df_previsao_agrupada = df_previsao_agrupada.sort_values(by='Posicao_Previsao')

In [ ]:
# Resolver empates considerando critérios de desempate
for i in range(len(df_previsao_agrupada)):
    time = df_previsao_agrupada.iloc[i]['Time']
    posicao = df_previsao_agrupada.iloc[i]['Posicao_Previsao']

    # Verificar empates
    empates = df_previsao_agrupada[df_previsao_agrupada['Posicao_Previsao'] == posicao]
    if len(empates) > 1:
        # Calcular o número de vitórias para cada time
        empates['Vitoria'] = dados_teste[dados_teste['Time'] == time]['Vitoria'].values[0]
        empates['Saldo_de_gols'] = dados_teste[dados_teste['Time'] == time]['Saldo_de_gols'].values[0]
        # Ordenar os times empatados pelos critérios de desempate
        empates = empates.sort_values(by=['Vitoria', 'Saldo_de_gols'], ascending=False)
        # Atribuir as posições aos times empatados
        for j in range(len(empates)):
            df_previsao_agrupada.loc[df_previsao_agrupada['Time'] == empates.iloc[j]['Time'], 'Posicao_Previsao'] = posicao + j

In [17]:
 # Exibir a previsão da colocação na 38ª rodada em uma tabela
print("Previsão da colocação na 38ª rodada:")
print(df_previsao_agrupada.head(20).reset_index(drop=True).to_string(index=False))

Previsão da colocação na 38ª rodada:
         Time  Posicao_Previsao
  Atlético-MG                 1
    Palmeiras                 2
     Flamengo                 3
       Grêmio                 4
     Botafogo                 5
   Bragantino                 6
 Athletico-PR                 8
   Fluminense                 9
       Cuiabá                10
    São Paulo                11
Internacional                12
    Fortaleza                13
     Cruzeiro                14
  Corinthians                15
        Vasco                16
        Bahia                17
        Goiás                18
     Coritiba                19
       Santos                20
   América-MG                21
